# Data Acquisition (ETL)

**Starting Point:**  
The article data originates from a Kafka datastream. It is not normalized (so it cannot be analyzed directly) and requires Active Directory login (making collaboration difficult).  
- [View Kafka topic data (AKHQ)](https://akhq.pdp.production.admin.srgssr.ch/ui/strimzi/topic/articles-v2/data?sort=NEWEST&partition=All)

- **Processing steps:**  
  1. Read article data from the Delta table populated from Kafka.
  2. Flatten and transform nested fields (e.g., titles, resources, contributors) using a SQL view.
  3. Create a Spark DataFrame from the flattened view and inspect the results.
  4. Write the DataFrame to a Delta table for analytics and automation.
  5. Export a <25MB Parquet sample with only public data for sharing (e.g., via GitHub).

**Goal:**  
The data should be available as a Parquet file for sharing. Since the dataset is large (5GB), only a public sample is exported for easy distribution.

**Access Control:**  
To guarantee data integrity and protect sensitive information, data distribution is based on user access rights. Entitled users can access the full confidential dataset, while restricted users are provided with only the public sample. This ensures that only authorized users can view sensitive data, maintaining compliance and data security.

In [0]:
%skip
# Run only once at the beginning of the analysis
# (to install all required libraries for translation and embeddings)

%pip install pandas==2.1.4
%pip install numpy==1.26.4
%pip install pyarrow==14.0.1
%pip install pyspark==4.0.0

%pip install sentence-transformers==5.1.2
%pip install transformers==4.48.1
%pip install torch==2.7.0
%pip install sentencepiece==0.1.99
%pip install safetensors==0.3.1
%pip install accelerate==0.24.0

%pip install scikit-learn==1.3.2
%pip install scipy==1.12.1

%restart_python

In [0]:
from pyspark.sql.utils import AnalysisException

def has_read_permission(table_name):
    try:
        spark.sql(f"SELECT 1 FROM {table_name} LIMIT 1")
        return True
    except AnalysisException:
        return False

has_read_access_udp_articles_v2 = has_read_permission("udp_prd_atomic.pdp.articles_v2")

## Step-01: Read from Kafka (SQL)

The following steps read article data from the Delta table `udp_prd_atomic.pdp.articles_v2`, which is assumed to be populated from a Kafka stream. The original Kafka data contains nested lists and complex structures (e.g., for multilingual fields or arrays of resources). In the transformation, the SQL view `articles_flat` flattens this nested data by extracting relevant fields and, where multiple values exist (such as for titles in different languages), selects the first available entry—typically prioritizing German (`'de'`) or otherwise the first value. This process prepares the data, along with Kafka metadata (key, topic, partition, offset, timestamp), for further analysis in a flat, tabular format.

In [0]:
%sql
-- Only run main logic if has_read_access_udp_articles_v2 is true

-- Uncomment below to run only when has_read_access_udp_articles_v2 = true
-- IF has_read_access_udp_articles_v2 THEN
CREATE OR REPLACE TEMP VIEW articles_flat AS
WITH base AS (
  SELECT
    key,
    topic,
    partition,
    offset,
    timestamp,
    value AS v
  FROM udp_prd_atomic.pdp.articles_v2
)
SELECT
  -- flat metadata
  v.id AS id,
  v.publisher AS publisher,
  v.provenance AS provenance,
  v.modificationDate AS modificationDate,
  v.releaseDate AS releaseDate,
  -- TITLE: always first entry
  try_element_at(transform(coalesce(v.title, array()), x -> x.content), 1) AS title_auto,
  -- LEAD: always first entry
  try_element_at(transform(coalesce(v.lead, array()), x -> x.content), 1) AS lead_auto,
  -- KICKER: always first entry
  try_element_at(transform(coalesce(v.kicker, array()), x -> x.content), 1) AS kicker_auto,
  -- Extract IDs specifically (not a Map!)
  try_element_at(
    transform(
      filter(coalesce(v.identifiers, array()), x -> x.type = 'urn'   AND x.value IS NOT NULL),
      x -> x.value
    ),
    1
  ) AS id_urn,
  try_element_at(
    transform(
      filter(coalesce(v.identifiers, array()), x -> x.type = 'srgId' AND x.value IS NOT NULL),
      x -> x.value
    ),
    1
  ) AS id_srg,
  -- First available image
  aggregate(
    coalesce(v.resources, array()),
    CAST(NULL AS STRING),
    (acc, r) -> coalesce(acc, r.picture.locator.url),
    acc -> acc
  ) AS picture_url,
  -- content.text.items -> CSV (space-separated)
  CASE
    WHEN v.content IS NOT NULL
      THEN concat_ws(' ', coalesce(v.content.text, CAST(array() AS ARRAY<STRING>)))
    ELSE NULL
  END AS content_text_csv,
  -- contributors -> CSV of names
  CASE
    WHEN v.contributors IS NOT NULL
      THEN concat_ws(
        ', ',
        transform(
          coalesce(v.contributors, array()),
          c -> coalesce(
            c.name,
            c.agent.person.name,
            c.agent.team.name,
            c.agent.department.name
          )
        )
      )
    ELSE NULL
  END AS contributors_csv,
  -- only first locator.url from resources -> CSV
  try_element_at(
    filter(
      flatten(
        transform(
          coalesce(v.resources, array()),
          r -> array(
            r.document.locator.url,
            r.picture.locator.url,
            r.link.locator.url
          )
        )
      ),
      x -> x IS NOT NULL
    ),
    1
  ) AS resources_locator_urls_csv,
  -- Keywords -> CSV
  CASE
    WHEN v.keywords IS NOT NULL
      THEN concat_ws(',', coalesce(v.keywords, CAST(array() AS ARRAY<STRING>)))
    ELSE NULL
  END AS keywords_csv,
  -- Kafka metadata (key safely as STRING)
  CAST(key AS STRING) AS key,
  topic,
  partition,
  offset,
  timestamp
FROM base;
-- END IF

## Step-02: Create DataFrame and Visually Inspect Results

The code below runs a Spark SQL query against the temporary view `articles_flat`, loads the result into a Spark DataFrame named `df`, and then displays the DataFrame for visual inspection. This step materializes the flattened article data so it can be further processed or written to a Delta table.

In [0]:
if has_read_access_udp_articles_v2:
    articles_flat = spark.sql("SELECT * FROM articles_flat").orderBy("modificationDate", ascending=False)
    display(articles_flat, truncate=False)

## Step-03: Save Data to Delta Table

In the next steps, the data will be saved both to a Delta table (for better automation and analytics).

### Write to (private) Delta Table - all articles

The following code appends the transformed DataFrame `df` to the Delta table `swi_audience_prd.pdp_articles_v2.articles_v2`. It writes in **append** mode, uses the **Delta** format, and enables **schema merging** so that any new columns are automatically added to the target table without overwriting existing data.

- Contains all articles (**confidential**)

In [0]:
if has_read_access_udp_articles_v2:
    articles_flat.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable("swi_audience_prd.pdp_articles_v2.articles_v2_flat")


### Write to (public) Parquet File - selected articles, manually Upload to GitHub

Export a <25 MB sample of the data with only public data as a Parquet file for easy sharing via GitHub.  
**Note:** The Parquet file must be manually downloaded from Databricks and then uploaded to your GitHub repository.

In [0]:
if has_read_access_udp_articles_v2:
    from pyspark.sql.functions import col, current_date
    import math
    import shutil
    import os

    # Estimate average row size in bytes using a sample of 100,000 rows
    sample_row = articles_flat.limit(100000).toPandas()
    avg_row_size = sample_row.memory_usage(index=True, deep=True).sum() / len(sample_row)

    # Set maximum file size to 25 MB
    max_bytes = 25 * 1024 * 1024  # 25 MB

    # Parquet is more efficient than CSV, so increase row count estimate by a factor of 2
    max_rows = int(math.floor((max_bytes / avg_row_size) * 2))

    # Filter articles to only those published (releaseDate today or earlier), then sample up to max_rows
    articles_flat_sampled = (
        articles_flat
        .filter(col("releaseDate") <= current_date())
        .limit(max_rows)
    )

    # Write the sampled DataFrame as a single Parquet file to a temporary directory
    parquet_temp_dir = "/Volumes/swi_audience_prd/pdp_articles_v2/pdp_articles_v2_volume/export_articles_v2_sample25mb_tmp_parquet"
    parquet_final_path = "/Volumes/swi_audience_prd/pdp_articles_v2/pdp_articles_v2_volume/export_articles_v2_sample25mb.parquet"

    (articles_flat_sampled.coalesce(1)
        .write
        .mode("overwrite")
        .parquet(parquet_temp_dir)
    )

    # Move the single Parquet part file to the final destination
    files = [f for f in os.listdir(parquet_temp_dir) if f.endswith(".parquet")]
    if files:
        src_file = os.path.join(parquet_temp_dir, files[0])
        shutil.move(src_file, parquet_final_path)

    # Remove the temporary directory after moving the file
    shutil.rmtree(parquet_temp_dir)

...now manually:

1. **Open the CSV file in Databricks:**
   - Navigate to [Databricks Volume Browser](https://adb-4119964566130471.11.azuredatabricks.net/explore/data/volumes/swi_audience_prd/pdp_articles_v2/pdp_articles_v2_volume?o=4119964566130471) in the Databricks workspace file browser.

2. **Download the file:**
   - Right-click on `export_articles_v2_sample25mb.parquet` and select **"Download"** to save the file to your local machine.

3. **Upload the file to GitHub:**
   - Go to [GitHub Folder](https://github.com/Tao-Pi/CAS-Applied-Data-Science/tree/main/Module-3/01_Module%20Final%20Assignment).
   - Click **"Add file"** > **"Upload files"**.
   - Drag and drop `export_articles_v2_sample25mb.parquet` or use the file picker to select it.
   - Commit the changes to upload the file.


## Step-04: Load Data Based on User Rights

The next step is to load the data, with access determined by user rights:

- **Restricted users** can load only the public data sample (e.g., the Parquet file exported for sharing).
- **Entitled users** can load the full, confidential dataset from the Delta table.

This ensures that sensitive information is only accessible to authorized users, while still allowing broader access to public data for collaboration and analysis.

In [0]:
import pandas as pd

# Restricted users 
url = "https://github.com/Tao-Pi/CAS-Applied-Data-Science/raw/main/Module-3/01_Module%20Final%20Assignment/export_articles_v2_sample25mb.parquet"
srgssr_article_corpus = pd.read_parquet(url)

# Entitled users
if has_read_access_udp_articles_v2:
    srgssr_article_corpus = spark.table("swi_audience_prd.pdp_articles_v2.articles_v2_flat")

In [0]:
if isinstance(srgssr_article_corpus, pd.DataFrame):
    srgssr_article_corpus = pd.DataFrame(srgssr_article_corpus)

# Dataset Overview

In this chapter, we provide a brief overview of the dataset used for analysis. We indicate whether the loaded dataset is the full confidential version or the public sample, report the total number of articles available, and present a first look at the articles data to understand its structure and content.


## Step 1: Check Dataset Version (Confidential vs Public)

Here we check and indicate whether the loaded dataset is the full confidential version or the public sample.

In [0]:
def format_rowcount(n):
    if n >= 1_000_000:
        return f"more than {n // 1_000_000} million"
    elif n >= 1_000:
        return f"more than {n // 1_000} thousand"
        return f"more than {n // 1_000_000} Mio."
    elif n >= 1_000:
        return f"more than {n // 1_000} Tsd."
    else:
        return f"{n}"

if has_read_access_udp_articles_v2:
    rowcount = srgssr_article_corpus.count()
    print(f"congrats: you have successfully read the full data set. This contains the full corpus of {format_rowcount(rowcount)} Articles published by SRG-SSR as plain text together with some relevant metadata. You can access the dataframe object by calling 'srgssr_article_corpus' from Python now.")
else:
    if isinstance(srgssr_article_corpus, pd.DataFrame):
        rowcount = len(srgssr_article_corpus)
    else:
        rowcount = srgssr_article_corpus.count()
    print(f"congrats: you have successfully read the publically available (sampled) data set. This contains an excerpt of {format_rowcount(rowcount)} articles within SRG-SSR as plain text together with some relevant metadata. You can access the dataframe object by calling 'srgssr_article_corpus' from Python now.")


## Step 2: Overview of the Data

In this step, we provide an overview of the data contained in the loaded dataset. This includes a summary of the available articles and a first look at their structure and content.

In [0]:
first_row = srgssr_article_corpus.head(1)[0].asDict() if srgssr_article_corpus.head(1) else {}
cols_info = [
    {
        "column": name,
        "type": str(dtype),
        "example": first_row.get(name, None)
    }
    for name, dtype in srgssr_article_corpus.dtypes
]

display(cols_info, ["column", "type", "example"])

## Step 3: A Closer Look

In this step, we take a deeper look at the loaded dataset, exploring its structure and content in more detail.

In [0]:
display(srgssr_article_corpus)

#Analyses
This is where analyses are performed. This is work in progress. Some ideas:

**Story 1:** I want to quickly search all existing articles without the need to use Google. I want to do that because when I write a story, I want to make sure the same story was not just written by my colleagues working in a different branch.

**Story 2:** I want to find out what topics SRG writes about – this could, for example, be used for navigation (News / Sport / etc.).

**Story 3:** I want to translate all existing articles into all languages used. This way, I can multiply the offer easily. Instead of having some articles in French and some in English, I will have all articles available in all of our 11 languages.

*List other ideas here...*

In [0]:
# ----------------------------------------------------------------------
# Sample the article corpus to a manageable size (1000 rows) for faster
# prototyping and to avoid OOM issues during embedding / clustering.
# This works for both Spark DataFrames and pandas DataFrames.
# ----------------------------------------------------------------------
import pandas as pd

if not isinstance(srgssr_article_corpus, pd.DataFrame):
    # Force convert Spark DataFrame to pandas DataFrame
    srgssr_article_corpus = srgssr_article_corpus.limit(100).toPandas()
else:
    # pandas: take the first 100 rows (or fewer if the DataFrame is smaller)
    srgssr_article_corpus = srgssr_article_corpus.head(100)

display(srgssr_article_corpus, truncate=False)

## USE CASE: Quickly Search All Existing Articles

I want to quickly search all existing articles without the need to use Google. I want to do that because when I write a story, I want to make sure the same story was not just written by my colleagues working in a different branch.

**Approach:**
- Implement a semantic search feature within Databricks that allows users to search articles by keywords, phrases, or topics.
- Use text embeddings (e.g., with Sentence Transformers) to represent article content and enable similarity-based search.
- Provide a simple search interface where users can enter queries and retrieve the most relevant articles.
- Optionally, add filters for date, author, or branch to refine search results.

In [0]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

# Load or reuse the embedder model
def get_embedder():
    if not hasattr(get_embedder, "model"):
        get_embedder.model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    return get_embedder.model

def add_embeddings(df):
    if "content_emb" not in df.columns:
        model = get_embedder()
        df["content_emb"] = model.encode(df["content_text_csv"].toPandas()["content_text_csv"].tolist(), show_progress_bar=False, batch_size=64).tolist()
    return df

def to_vector(arr):
    return np.array(arr)

srgssr_article_corpus = add_embeddings(srgssr_article_corpus)
srgssr_article_corpus["content_vec"] = srgssr_article_corpus["content_emb"].apply(to_vector)

def semantic_search(query, top_k=10):
    model = get_embedder()
    query_emb = model.encode([query])[0]
    query_vec = np.array(query_emb)
    def cosine_similarity(vec):
        return float(np.dot(vec, query_vec) / (np.linalg.norm(vec) * np.linalg.norm(query_vec) + 1e-10))
    srgssr_article_corpus["similarity"] = srgssr_article_corpus["content_vec"].apply(cosine_similarity)
    results = srgssr_article_corpus.sort_values("similarity", ascending=False).head(top_k)[["id", "content_text_csv", "similarity"]]
    display(results)

# Example usage: search for a topic/phrase/keyword
semantic_search("climate change", top_k=10)


##USE CASE: find out what topics SRG writes about.
**Approach:**  
- Read the text from the `content_text_csv` column of the articles.
- Compute similarity between article contents, e.g., by embedding the texts and using a Random Forest or other clustering/classification methods to group similar articles.
- Identify clusters of similar content to reveal common topics or themes.
- Use these clusters to enhance navigation and filtering options for users.

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Assume 'articles_with_emb' DataFrame exists and has 'content_vec' and a label column 'label_col'
# If no label column exists, create a dummy one for demonstration
if "label_col" not in articles_with_emb.columns:
    from pyspark.sql.functions import monotonically_increasing_id
    articles_with_emb = articles_with_emb.withColumn("label_col", (monotonically_increasing_id() % 2).cast("double"))

# Index label column if not already numeric
label_indexer = StringIndexer(inputCol="label_col", outputCol="label")
df_indexed = label_indexer.fit(articles_with_emb).transform(articles_with_emb)

# Assemble features (already in 'content_vec')
assembler = VectorAssembler(inputCols=["content_vec"], outputCol="features")
df_features = assembler.transform(df_indexed)

# Split into train/test
train, test = df_features.randomSplit([0.8, 0.2], seed=42)

# Train Random Forest
clf = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=20, seed=42)
model = clf.fit(train)

# Predict
predictions = model.transform(test)

# Evaluate
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Random Forest accuracy:", accuracy)
print("Random Forest params:", clf.extractParamMap())

In [0]:
# For quick prototyping, we sample only 1000 articles here.
# To run on the full dataset, remove the .limit(1000) line below.
df_sample = srgssr_article_corpus.limit(1000)

from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import ArrayType, FloatType
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.clustering import KMeans
import pandas as pd

# Load model once per executor for efficiency
from sentence_transformers import SentenceTransformer

def get_embedder():
    if not hasattr(get_embedder, "model"):
        get_embedder.model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    return get_embedder.model

@pandas_udf(ArrayType(FloatType()))
def embed_udf(texts: pd.Series) -> pd.Series:
    model = get_embedder()
    return pd.Series(model.encode(texts.tolist(), show_progress_bar=False, batch_size=64).tolist())

# Add embeddings column (no translation)
df_emb = df_sample.withColumn(
    "content_emb", embed_udf(col("content_text_csv"))
)

# Convert array to Spark Vector (required by ML algorithms)
to_vector = pandas_udf(lambda arr: [Vectors.dense(x) if x else Vectors.dense([]) for x in arr], returnType=VectorUDT())
df_vec = df_emb.withColumn("features", to_vector(col("content_emb")))

# KMeans clustering (choose number of clusters, e.g., 10)
k = 10
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=k, seed=42)
model = kmeans.fit(df_vec)

# Assign cluster IDs
df_clustered = model.transform(df_vec).select("id", "cluster", "content_text_csv")

# Show a sample of clustered articles
display(df_clustered.orderBy("cluster"))


## USE CASE: Translate All Existing Articles into All Languages Used

Goal: Automatically translate every article into all supported languages, so each article is available in every language used by SRG.

**Approach:**
- For each article, use the `ai_translate` function to generate translations for all target languages.
- Store the translated articles alongside the originals for easy access and analytics.

**Example (SQL):**
sql
SELECT
  *,
  ai_translate(title, 'fr') AS title_fr,
  ai_translate(title, 'it') AS title_it,
  ai_translate(title, 'en') AS title_en,
  ai_translate(title, 'rm') AS title_rm
FROM articles_flat

*(Repeat for all relevant text fields and all required languages.)*

In [0]:
# Assume ai_translate(text: str, target_lang: str) -> str is defined and available

from pyspark.sql.functions import col, struct, udf
from pyspark.sql.types import StringType

target_languages = ["en", "fr", "it", "de"]  # Example target languages

def make_translate_udf(lang):
    @udf(StringType())
    def translate_udf(text):
        return ai_translate(text, lang)
    return translate_udf

df = srgssr_article_corpus

for lang in target_languages:
    df = df.withColumn(f"content_text_csv_{lang}", make_translate_udf(lang)(col("content_text_csv")))

# Store the DataFrame with translations alongside originals
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("swi_audience_prd.pdp_articles_v2.articles_v2_translated")

display(df)

In [0]:
# Falls noch nicht installiert+neu gestartet:
# %pip install -U transformers sentencepiece torch safetensors accelerate
# dbutils.library.restartPython()

from transformers import pipeline
import pandas as pd

# 1) 20 Zeilen holen (nur nötige Spalten)
pdf = (
    srgssr_article_corpus
    .select("id", "content_text_csv")
    .limit(20)
    .toPandas()
)

# 2) Pipeline lokal laden (CPU)
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-mul-en", device=-1)

# 3) Übersetzen (Batch pro Text-Chunk – hier simpel ohne Chunking)
def translate_text(t):
    if t is None or not str(t).strip():
        return None
    res = translator(t, truncation=True, max_length=1000)
    return res[0]["translation_text"]

pdf["content_text_csv_english"] = [translate_text(t) for t in pdf["content_text_csv"]]

# 4) Zurück nach Spark (nur für Anzeige)
df_test = spark.createDataFrame(pdf)
display(df_test)

In [0]:
%skip
# 🧪 Nur 20 Zeilen nehmen
df_sample = srgssr_article_corpus.limit(20)

# 🔄 Übersetzungsspalte hinzufügen
df_test = df_sample.withColumn(
    "content_text_csv_english",
    translate_series_to_en(F.col("content_text_csv"))
)

# 👀 Ergebnis anzeigen
df_test.select("id", "content_text_csv", "content_text_csv_english").display()

#Hints and Notes
> **Hint:**  
> For a temporary Jupyter environment to experiment or explore data, consider using [RenkuLab](https://renkulab.io/p/snsf-anoxia-project/proxy-proxy/sessions/01JX2TG1RZ9J0PQ53H3RT81BD4/start). RenkuLab offers cloud-based notebook sessions—no local setup required.

> **Note:**  
> RenkuLab sessions may require authentication and have limited resources. Save your work frequently, as sessions can time out or be terminated after inactivity.